# **Programa conexion **
aqui se procede a montar nuestro drive para poder guardar nuestro repositorio donde trabajaremos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Creamos un espacio de trabajo en google drive, luego de autorizar creamos nuestra ruta de destino y clonamos el repositorio donde se encuentra todo los archivos



In [ ]:
import os
# Ruta donde guardaras el repositorio
ruta_destino= '/content/drive/MyDrive/MiRepoMareografico'
#Creamos la carpeta si no existe
os.makedirs(ruta_destino,exist_ok=True)

# Reemplaza la URL por el repositorio que quieras
url_repo='https://github.com/Octavio341/PruebaconGoogleColab.git'

#cambiar al directorio de destino
%cd $ruta_destino
#Clonar
!git clone $url_repo

/content/drive/MyDrive/MiRepoMareografico
Cloning into 'PruebaconGoogleColab'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 144 (delta 51), reused 77 (delta 31), pack-reused 0 (from 0)
Receiving objects: 100% (144/144), 877.08 KiB | 11.10 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files, drive
import ipywidgets as widgets
from IPython.display import display, clear_output
##################################################
##################  NUEVAS IMPORTACIONES #########
##################################################
import datetime
import numpy as np
from os.path import basename
from pathlib import Path

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool,CrosshairTool,DatetimeTickFormatter, LabelSet,ColumnDataSource
from scipy.interpolate import make_interp_spline




#graficar linea

#############################################################################

archivos_cargados = [] # esto nos sirve para guardar los nombres de los archivos
lista_archivos=[]

#widgets
# Widgets principales
output = widgets.Output()

# Título
titulo = widgets.HTML("<h3 style='color: teal;'>Selecciona el origen del archivo</h3>")

# Selector de origen
tipo_selector = widgets.ToggleButtons(
    options=['Subir desde PC', 'Usar desde Drive'],
    description='Origen:',
    button_style='info', # colores: 'primary', 'success', 'info', 'warning', 'danger'
    style={'description_width': 'initial'}
)

# Widget para subir archivos desde PC
upload_widget = widgets.FileUpload(
    accept='',  # acepta cualquier archivo
    multiple=False,
    description='📂 Subir archivo',
    button_style='success'
)

#Carpeta inicial en Drive
ruta_drive = '/content/drive/MyDrive'
ruta_actual = [ruta_drive]#lista para navegar hacia atras facilmente


#Mostrar lista de archivos y carpetas
def mostrar_archivos_y_graficar():
  clear_output(wait=True)
  archivos = os.listdir(ruta_actual[0])
  archivos.sort()


  botones = []
  if ruta_actual[0] != ruta_drive:
    boton_volver = widgets.Button(description='<-- Volver atras',button_style='warning')
    boton_volver.on_click(volver_atras)
    botones.append(boton_volver)

  for archivo in archivos:
    ruta_completa = os.path.join(ruta_actual[0],archivo)
    if os.path.isdir(ruta_completa):
      b = widgets.Button(description=f"{archivo}",layout=widgets.Layout(width='auto'))
      b.on_click(lambda x,rc=ruta_completa: entrar_carpeta(rc))
      botones.append(b)
    elif archivo.endswith('.dat'):
      b = widgets.Button(description =f"{archivo}",layout=widgets.Layout(width='auto'))
      b.on_click(lambda x,rc =ruta_completa:graficar_archivo(rc) )
      botones.append(b)
  display(titulo, tipo_selector,widgets.VBox(botones))

# Volver a carpeta anterior
def entrar_carpeta(ruta):
  ruta_actual[0]=ruta
  mostrar_archivos_y_graficar()

def volver_atras(b=None):
  if ruta_actual[0] != ruta_drive:
    ruta_actual[0] = os.path.dirname(ruta_actual[0])
  mostrar_archivos_y_graficar()

#Leer y graficar archivo .dat

def graficar_archivo(ruta_archivo):
    titulo2 = widgets.HTML("<h3 style='color: teal;'>Analisis</h3>")
    display(titulo2)
    # Mostrar interfaz inicial
    lista_archivos = []
    if ruta_archivo: # esto nos sirve para preguntar si se ha seleccionado un archivo
      lista_archivos.append(Path(ruta_archivo).name)

      for nombre_archivo in lista_archivos:
        ya_registrado = False # esto es nuestra bandera, para saber si el archivo a sido agregado

        for nombre in archivos_cargados:
          if nombre_archivo == nombre:
            print("----------------------------------------------------------------------------------------")
            print(f"======  El archivo '{nombre_archivo}'ya ha se ha registrado")
            ya_registrado = True  # marcar como duplicado
            break

        if not ya_registrado:
          archivos_cargados.append(nombre_archivo)
          print("archivos cargados",archivos_cargados)
          matriz_toga = pd.read_csv(ruta_archivo,sep=r'\s+',names =["StationID", "StationName", "Date", "D1", "D2", "D3","D4", "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12"], engine = 'python', skiprows = 1, na_values = "9999")
          #print("Matriz toga",matriz_toga.head())
          # version mejorada de google colab

          display(matriz_toga.head(10)) # podemos ponerle valore al head(10) conforme a las filas que queremos ver
          lista_fecha= list()
          lista_datos=list()

          for ind in matriz_toga.index:
            fecha = str(matriz_toga["Date"][ind])
            if fecha[8:9]== "1":
              fecha_inicial = datetime.datetime(int(fecha[:4]),int(fecha[4:6]),int(fecha[6:8]))
            else:
              fecha_inicial = datetime.datetime(int(fecha[:4]),int(fecha[4:6]),int(fecha[6:8]),12)
            for dat in ["D1", "D2", "D3", "D4",  "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12"]:
              lista_fecha.append(fecha_inicial)
              lista_datos.append(matriz_toga[dat][ind])
              fecha_inicial=fecha_inicial+datetime.timedelta(hours=1)

          output_notebook()  # o usa output_file("archivo.html") si estás fuera de notebook
          #crear herramientas

          herramientas=[
              "pan",
              "wheel_zoom",
              "box_zoom",
              "reset",
              "save",
              "lasso_select",
              "box_select",
              "tap",
              "zoom_in",
              "zoom_out",
              "help"
          ]
          p = figure(
              title=f"Gráfica del archivo Toga: {lista_archivos}",
              x_axis_label="Fecha",
              y_axis_label="Nivel del mar (mm)",
              x_axis_type="datetime",
              width=1000,
              height=600,
              tools=','.join(herramientas),
              toolbar_location="above",
              toolbar_sticky=False,
              margin=(50, 50, 50, 50)
          )
          #Añadir HoverTool(muestra tooltip con datos exactos al pasar el mouse)
          hover = HoverTool(
              tooltips=[("Fecha", "@x{%F}"), ("Nivel", "@y")],
              formatters={'@x': 'datetime'},
              mode='vline'
          )

          #Añadir CrosshairTool (Muestra lineas guia horizontal y verticales para facilitar lectura)
          crosshair = CrosshairTool(dimensions= 'both')#lineas vertical y horizontal
          p.add_tools(crosshair)

          p.line(lista_fecha, lista_datos, line_width=2, color="blue")
          p.title.text_font_size = "20pt"
          p.xaxis.axis_label_text_font_size = "16pt"
          p.yaxis.axis_label_text_font_size = "16pt"
          p.add_tools(HoverTool(tooltips=[("Fecha", "@x{%F}"), ("Nivel", "@y")], formatters={'@x': 'datetime'}))
          p.toolbar.logo = None  # Ocultar logo de Bokeh
          show(p)


    else:
      print("No se ha seleccionado ningún archivo.")
# Manejador de archivos locales
def manejar_archivo_local(change):
    for filename, fileinfo in upload_widget.value.items():
        content = fileinfo['content']
        with open(filename, 'wb') as f:
            f.write(content)
        lista_archivos.append(Path(filename).name)
        graficar_archivo(filename)

# Manejador para cambiar entre PC y Drive
def cambiar_origen(change):
    with output:
        clear_output(wait=True)
        if tipo_selector.value == 'Subir desde PC':
            display(upload_widget)
        else:
            mostrar_archivos_y_graficar()
# Observador del cambio de origen
tipo_selector.observe(cambiar_origen, names='value')

# Vincular carga con función solo una vez
upload_widget.observe(manejar_archivo_local, names='value')
display(widgets.VBox([titulo, tipo_selector, output]))


In [ ]:
def graficar_archivo():
    from bokeh.plotting import figure
    p = figure(title="Ejemplo sin botón", width=400, height=300)
    p.line([1,2,3], [4,6,2], line_width=2)
    return p

from bokeh.plotting import output_notebook, show
from bokeh.layouts import column
from bokeh.models import Button, CustomJS

output_notebook()

# Pre-generamos la figura
p = graficar_archivo()
p.visible = False  # Oculta la figura inicialmente

# Creamos un botón
boton = Button(label="Mostrar/Ocultar gráfica", button_type="success")

# Con JavaScript cambiamos la visibilidad de la figura
boton.js_on_click(CustomJS(args=dict(fig=p), code="""
    fig.visible = !fig.visible;
"""))

# Layout con botón y figura
layout = column(boton, p)

show(layout)



In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column, row
from bokeh.models import Button, CustomJS

output_notebook()

def crear_layout():
    # Gráfica de ejemplo inicialmente oculta
    p = figure(title="Gráfica ejemplo", width=400, height=300, visible=False)
    p.line([1, 2, 3], [4, 6, 2], line_width=2)

    # Botón para mostrar u ocultar la gráfica
    boton = Button(label="Mostrar / Ocultar gráfica", button_type="primary", width=200)

    # Callback JS para alternar visibilidad de la gráfica
    boton.js_on_click(CustomJS(args=dict(fig=p), code="""
        fig.visible = !fig.visible;
    """))

    # Controles X (zoom in/out)
    boton_x_in = Button(label="Zoom In X")
    boton_x_out = Button(label="Zoom Out X")

    boton_x_in.js_on_click(CustomJS(args=dict(p=p), code="""
        let start = p.x_range.start;
        let end = p.x_range.end;
        let center = (start + end) / 2;
        let new_range = (end - start) * 0.8;
        p.x_range.start = center - new_range / 2;
        p.x_range.end = center + new_range / 2;
    """))

    boton_x_out.js_on_click(CustomJS(args=dict(p=p), code="""
        let start = p.x_range.start;
        let end = p.x_range.end;
        let center = (start + end) / 2;
        let new_range = (end - start) * 1.2;
        p.x_range.start = center - new_range / 2;
        p.x_range.end = center + new_range / 2;
    """))

    controles_x = row(boton_x_in, boton_x_out)

    # Controles Y (zoom in/out)
    boton_y_in = Button(label="Zoom In Y")
    boton_y_out = Button(label="Zoom Out Y")

    boton_y_in.js_on_click(CustomJS(args=dict(p=p), code="""
        let start = p.y_range.start;
        let end = p.y_range.end;
        let center = (start + end) / 2;
        let new_range = (end - start) * 0.8;
        p.y_range.start = center - new_range / 2;
        p.y_range.end = center + new_range / 2;
    """))

    boton_y_out.js_on_click(CustomJS(args=dict(p=p), code="""
        let start = p.y_range.start;
        let end = p.y_range.end;
        let center = (start + end) / 2;
        let new_range = (end - start) * 1.2;
        p.y_range.start = center - new_range / 2;
        p.y_range.end = center + new_range / 2;
    """))

    controles_y = column()

    # Layout final: botón principal arriba, gráfica y controles abajo
    return column(boton, p, controles_x, controles_y)

layout_completo = crear_layout()
show(layout_completo)


In [ ]:
from bokeh.layouts import column
from bokeh.models import Button, CustomJS, ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.io import output_file

# Archivo HTML de salida
output_file("boton_bokeh.html")

# Fuente de datos
source = ColumnDataSource(data=dict(x=[1, 2, 3, 4, 5], y=[6, 7, 2, 4, 5]))

# Gráfica
p = figure(title="Gráfica de ejemplo", width=400, height=300)
line = p.line('x', 'y', source=source, line_width=2, line_color='blue')
p.visible = False  # Ocultamos la gráfica al principio

# Botón para mostrar/ocultar
boton = Button(label="Mostrar/Ocultar Gráfica", button_type="success")
boton.js_on_click(CustomJS(args=dict(figura=p), code="""
    figura.visible = !figura.visible
"""))

# Mostrar en navegador
show(column(boton, p))


In [ ]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column
import ipywidgets as widgets
from IPython.display import display, clear_output

output_notebook()

# Función que genera la gráfica, puede usar datos o no
def graficar_archivo():
    p = figure(title="Ejemplo", width=400, height=300)
    # Aquí decides si hay datos o no
    datos = [1, 2, 3]
    if datos:
        p.line([1, 2, 3], [4, 6, 2], line_width=2)
    else:
        p.text(1, 1, text=["No hay datos para mostrar"], text_font_size="16pt", text_color="red")
    return p

# Contenedor para mostrar la gráfica
output = widgets.Output()

# Función que se ejecuta al presionar el botón
def on_button_clicked(b):
    with output:
        clear_output(wait=True)  # Limpia salida previa
        p = graficar_archivo()
        show(p)

# Crear botón con ipywidgets (permite llamar funciones Python en Colab)
boton = widgets.Button(description="Mostrar gráfica", button_style='success')
boton.on_click(on_button_clicked)

# Mostrar botón y contenedor
display(boton, output)


Button(button_style='success', description='Mostrar gráfica', style=ButtonStyle())

Output()

In [ ]:
from bokeh.plotting import figure, output_notebook, show
from IPython.display import display, clear_output
import ipywidgets as widgets

output_notebook()



# Función para mostrar/ocultar la gráfica
def mostrar_u_ocultar(b=None):
    global grafica_visible
    clear_output(wait=True)

    if grafica_visible:
        grafica_visible = False
        print("Gráfica oculta.")
    else:
        grafica_visible = True
        x = [1, 2, 3, 4, 5]
        y = [2, 4, 6, 8, 10]
        p = figure(title="Gráfica Bokeh visible", x_axis_label='Eje X', y_axis_label='Eje Y')
        p.line(x, y, line_width=2)
        show(p)

    display(boton)  # Volver a mostrar el botón

# Crear botón
boton = widgets.Button(description="Mostrar / Ocultar gráfica")
boton.on_click(mostrar_u_ocultar)

# Mostrar el botón inicialmente
display(boton)


Gráfica oculta.


Button(description='Mostrar / Ocultar gráfica', style=ButtonStyle())

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.plotting import figure
from bokeh.models import Button, Select
from bokeh.events import ButtonClick
import random

output_notebook()

# Selector simulado
select_archivo = Select(title="Archivo", value="", options=["", "archivo1.txt", "archivo2.txt"])

# Botón
boton_graficar = Button(label="Graficar", button_type="success")

# Output layout inicial
output_layout = column()

# Función para graficar si se selecciona algo
def graficar_archivo():
    archivo = select_archivo.value
    if archivo:
        p = figure(title=f"Gráfico para {archivo}", width=400, height=300)
        p.line([1, 2, 3], [random.randint(1, 10) for _ in range(3)], line_width=2)
        return p
    else:
        print("⚠️ No se ha seleccionado ningún archivo.")
        return None

# Función que se llama al presionar el botón
def al_hacer_click(evento):
    resultado = graficar_archivo()
    if resultado:
        show(column(select_archivo, boton_graficar, resultado))
    else:
        show(column(select_archivo, boton_graficar))

# Asociar botón al evento
boton_graficar.on_event(ButtonClick, al_hacer_click)

# Mostrar controles iniciales
show(column(select_archivo, boton_graficar))



You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/js_callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In [ ]:
anio= 2000
def es_bisiesto(anio):
  resto = anio % 4
  if resto == 0:
    resto = anio % 100
    if resto == 0:
      resto = anio % 400
      if resto == 0:
          print("bisiesto")
          return True # divisible entre 4 - 100 - 400 es bisiesto
      else:
        return False
    else:
      print("bisiesto")
      return True # divisible entre 4 pero no entre 100 es bisiesto
  else:
    return False

print(f"el año{anio}{es_bisiesto(anio)}")



bisiesto
el año2000True


In [ ]:
############################################################################## divisor
#==== explicacion  ============

## TITULO ##
# DEFINICION GENERAL DE SU FUNCION


IMPLEMENTACION DEFINITIVA

In [ ]:
import ipywidgets as widgets # no permite trabajar con jupyter
from IPython.display import display, HTML,Image,YouTubeVideo

def crear_pestaña():

    fixed_height = '2060px' # establecemos un ALTURA fija para los contenidos
    fixed_width = '1400px' # establecemos un ANCHO fijo para los contenidos
    #le pasamos un los parametros a nuestros widgets
    inicio = widgets.Output(layout=widgets.Layout(height=fixed_height,width=fixed_width))
    imagen = widgets.Output(layout=widgets.Layout(heigth=fixed_height,width=fixed_width))
    html = widgets.Output(layout=widgets.Layout(heigth=fixed_height,width=fixed_width))
    video = widgets.Output(layout=widgets.Layout(heigth=fixed_height,width=fixed_width))
    #==== explicacion ======
    # les damos un estilo unico y fijo
    #=================================

    #ENVIAMOS CONTENIDO DENTRO DEL WIDGET
    with inicio:
      display(widgets.HTML("<h2 style= 'color:blue;'>Bienvenido</h2><p>esto es texto</p>"))
    with imagen:
      display(Image("https://i.ytimg.com/vi/GP-EWh7Qnuo/maxresdefault.jpg"))
    with html:
      display(HTML("<div syle='color:green;font-size:18px;'>Esto es un bloque de<b>HTML</b>con color verde.</div>"))
    with video:
      display(YouTubeVideo(''))
    # LAS PESTAÑAS DONDE ESTARAN LA MAYORIA DE LOS CONTENIDOS
    tab = widgets.Tab(children=[inicio,imagen,html,video])
    tab.set_title(0,'Inicio')
    tab.set_title(1,'Imagen')
    tab.set_title(2,'HTML')
    tab.set_title(3,'Video')

    ventana_estilizada = widgets.Box([tab],layout=widgets.Layout(
    # caja personalizada
        width= '2000px',#ancho del widgets
        height='500px',# definemos la altura del contendor
  ))

    # Añadir la clase css al box para aplicar estilos (modificamos clases en la salida HTML)
    ventana_estilizada.add_class('widget-box')# nombre de la clase css
    return ventana_estilizada # de vuelve un widget

ventana_principal= crear_pestaña() #llamamos nuestra funcion
display(ventana_principal)# renderiza el widget en la celda de salida

boton_nueva_ventana = widgets.Button(description="Abrir otra ventana")#Guardar el boton interactivo creado
output_ventana = widgets.Output()# se pueden mostrar otra ventana al hacer clic

def abrir_otra_ventana(b): # esto es para cuando hagamos un clic
# cuando hagmos el clic
  with output_ventana: # todo contenido que se imprime dentro
    nueva_ventana = crear_pestaña()# devuelve un box con pestañas o widgets interactivos
    display(nueva_ventana)# se vera debajo del boton

boton_nueva_ventana.on_click(abrir_otra_ventana)
display(boton_nueva_ventana,output_ventana)



Box(children=(Tab(children=(Output(layout=Layout(height='2060px', width='1400px')), Output(layout=Layout(width…

Button(description='Abrir otra ventana', style=ButtonStyle())

Output()

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# === Crear los widgets compartidos ===
numero1 = widgets.IntText(description='Número 1:')
numero2 = widgets.IntText(description='Número 2:')
boton_sumar = widgets.Button(description='Sumar')

resultado = widgets.Output()

# === Pestaña 1: entrada de datos ===
pestana_entrada = widgets.VBox([numero1, numero2, boton_sumar])

# === Pestaña 2: muestra resultado ===
pestana_resultado = widgets.VBox([widgets.HTML("<b>Resultado:</b>"), resultado])

# === Función de sincronización ===
def hacer_suma(b):
    with resultado:
        clear_output()
        suma = numero1.value + numero2.value
        print(f"La suma de {numero1.value} + {numero2.value} es {suma}")

boton_sumar.on_click(hacer_suma)

# === Crear tabs principales ===
tabs = widgets.Tab(children=[pestana_entrada, pestana_resultado])
tabs.set_title(0, 'Entrada')
tabs.set_title(1, 'Resultado')

display(tabs)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# ==== Función para crear una ventana independiente con pestañas ====
def crear_ventana_con_tabs():
    # Widgets únicos para esta ventana
    numero1 = widgets.IntText(description='Número 1:')
    numero2 = widgets.IntText(description='Número 2:')
    boton_sumar = widgets.Button(description='Sumar')
    resultado = widgets.Output()

    # Pestaña 1: Entrada
    pestana_entrada = widgets.VBox([numero1, numero2, boton_sumar])

    # Pestaña 2: Resultado
    pestana_resultado = widgets.VBox([widgets.HTML("<b>Resultado:</b>"), resultado])

    # Función para manejar la suma (solo afecta a estos widgets)
    def hacer_suma(b):
        with resultado:
            clear_output()
            suma = numero1.value + numero2.value
            print(f"La suma de {numero1.value} + {numero2.value} es {suma}")

    boton_sumar.on_click(hacer_suma)

    # Crear y devolver las pestañas
    tabs = widgets.Tab(children=[pestana_entrada, pestana_resultado])
    tabs.set_title(0, 'Entrada')
    tabs.set_title(1, 'Resultado')
    return tabs

# Botón para generar nuevas ventanas
boton_nueva = widgets.Button(description="Abrir nueva ventana")
salida_ventanas = widgets.Output()

def abrir_nueva_ventana(b):
    with salida_ventanas:
        display(crear_ventana_con_tabs())

# Asignar el evento al botón
boton_nueva.on_click(abrir_nueva_ventana)

# Mostrar botón y contenedor de nuevas ventanas
display(boton_nueva, salida_ventanas)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# ==== Función para crear una ventana con pestañas y su propio botón ====
def crear_ventana_con_tabs_y_boton():
    # Widgets únicos para esta ventana
    numero1 = widgets.IntText(description='Número 1:')
    numero2 = widgets.IntText(description='Número 2:')
    boton_sumar = widgets.Button(description='Sumar')
    resultado = widgets.Output()

    # Pestaña 1: Entrada
    pestana_entrada = widgets.VBox([numero1, numero2, boton_sumar])

    # Pestaña 2: Resultado
    pestana_resultado = widgets.VBox([widgets.HTML("<b>Resultado:</b>"), resultado])

    # Acción para sumar
    def hacer_suma(b):
        with resultado:
            clear_output()
            suma = numero1.value + numero2.value
            print(f"La suma de {numero1.value} + {numero2.value} es {suma}")

    boton_sumar.on_click(hacer_suma)

    # Crear las pestañas
    tabs = widgets.Tab(children=[pestana_entrada, pestana_resultado])
    tabs.set_title(0, 'Entrada')
    tabs.set_title(1, 'Resultado')

    # Crear un nuevo botón asociado a esta ventana
    boton_nuevo = widgets.Button(description="Abrir nueva ventana")
    contenedor = widgets.VBox([tabs, boton_nuevo])  # Contenedor conjunto

    def abrir_otro(b):
        nueva_ventana = crear_ventana_con_tabs_y_boton()
        salida_ventanas.children += (nueva_ventana,)

    boton_nuevo.on_click(abrir_otro)

    return contenedor

# Contenedor general donde se agregan todas las ventanas
salida_ventanas = widgets.VBox()

# Agregar la primera ventana con su botón
primera_ventana = crear_ventana_con_tabs_y_boton()
salida_ventanas.children += (primera_ventana,)

# Mostrar todo
display(salida_ventanas)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Contenedor principal
contenedor_principal = widgets.VBox()
salida_ventanas = widgets.VBox()

# === Función para crear una ventana con pestañas y botón de cerrar ===
def crear_ventana_con_tabs():
    numero1 = widgets.IntText(description='Número 1:')
    numero2 = widgets.IntText(description='Número 2:')

    boton_sumar = widgets.Button(description='Sumar')
    resultado = widgets.Output()

    # Contenedor de resultado
    pestana_entrada = widgets.VBox([numero1, numero2, boton_sumar])
    pestana_resultado = widgets.VBox([widgets.HTML("<b>Resultado:</b>"), resultado])

     # Función para manejar la suma (solo afecta a estos widgets)
    def hacer_suma(b):
        with resultado:
            clear_output()
            suma = numero1.value + numero2.value
            print(f"La suma de {numero1.value} + {numero2.value} es {suma}")

            from bokeh.plotting import figure, show, output_notebook

            output_notebook()

            p = figure(title="Gráfica de ejemplo con Bokeh", x_axis_label='X', y_axis_label='Y')

            x = [1, 2, 3, 4, 5]
            y = [6, 7, 2, 4, 5]

            p.line(x, y, legend_label="Línea", line_width=2)
            p.scatter(x, y, size=8, color="red", marker="circle", legend_label="Puntos")

            show(p)


    boton_sumar.on_click(hacer_suma)

    tabs = widgets.Tab(children=[pestana_entrada, pestana_resultado])
    tabs.set_title(0, 'Entrada')
    tabs.set_title(1, 'Resultado')

    # Botón para cerrar esta ventana
    boton_cerrar = widgets.Button(description="❌ Cerrar ventana", button_style='danger')

    contenedor_ventana = widgets.VBox([tabs, boton_cerrar])

    def cerrar_ventana(b):
        salida_ventanas.children = tuple(child for child in salida_ventanas.children if child != contenedor_ventana)

    boton_cerrar.on_click(cerrar_ventana)

    return contenedor_ventana

# === Función para abrir una nueva ventana ===
def abrir_nueva_ventana(b):
    nueva_ventana = crear_ventana_con_tabs()
    salida_ventanas.children = list(salida_ventanas.children) + [nueva_ventana]
    # Al final se asegura que el botón quede al final del todo
    contenedor_principal.children = [salida_ventanas, boton_nueva]

# Botón para nueva ventana
boton_nueva = widgets.Button(description="➕ Abrir nueva ventana", button_style='info')
boton_nueva.on_click(abrir_nueva_ventana)

# Mostrar interfaz inicial
contenedor_principal.children = [salida_ventanas, boton_nueva]
display(contenedor_principal)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from bokeh.plotting import figure
from bokeh.embed import components

# Contenedor principal
contenedor_principal = widgets.VBox()
salida_ventanas = widgets.VBox()

def crear_ventana_con_tabs():
    numero1 = widgets.IntText(description='Número 1:')
    numero2 = widgets.IntText(description='Número 2:')

    boton_sumar = widgets.Button(description='Sumar')
    resultado = widgets.Output()

    pestana_entrada = widgets.VBox([numero1, numero2, boton_sumar])
    pestana_resultado = widgets.VBox([widgets.HTML("<b>Resultado:</b>"), resultado])

    def hacer_suma(b):
        with resultado:
            clear_output()
            suma = numero1.value + numero2.value
            print(f"La suma de {numero1.value} + {numero2.value} es {suma}")

            # Crear gráfica con Bokeh
            p = figure(title="Gráfica de ejemplo con Bokeh", x_axis_label='X', y_axis_label='Y')
            x = [1, 2, 3, 4, 5]
            y = [6, 7, 2, 4, 5]
            p.line(x, y, legend_label="Línea", line_width=2)
            p.scatter(x, y, size=8, color="red", marker="circle", legend_label="Puntos")

            # Extraer script y div para embebido
            script, div = components(p)

            # Mostrar en salida como HTML
            display(HTML(div + script))

    boton_sumar.on_click(hacer_suma)

    tabs = widgets.Tab(children=[pestana_entrada, pestana_resultado])
    tabs.set_title(0, 'Entrada')
    tabs.set_title(1, 'Resultado')

    boton_cerrar = widgets.Button(description="❌ Cerrar ventana", button_style='danger')

    contenedor_ventana = widgets.VBox([tabs, boton_cerrar])

    def cerrar_ventana(b):
        salida_ventanas.children = tuple(child for child in salida_ventanas.children if child != contenedor_ventana)

    boton_cerrar.on_click(cerrar_ventana)

    return contenedor_ventana

def abrir_nueva_ventana(b):
    nueva_ventana = crear_ventana_con_tabs()
    salida_ventanas.children = list(salida_ventanas.children) + [nueva_ventana]
    contenedor_principal.children = [salida_ventanas, boton_nueva]

boton_nueva = widgets.Button(description="➕ Abrir nueva ventana", button_style='info')
boton_nueva.on_click(abrir_nueva_ventana)

contenedor_principal.children = [salida_ventanas, boton_nueva]
display(contenedor_principal)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook

output_notebook()  # Para mostrar Bokeh en notebook

contenedor_principal = widgets.VBox()
salida_ventanas = widgets.VBox()

def crear_ventana_con_tabs():
    numero1 = widgets.IntText(description='Número 1:')
    numero2 = widgets.IntText(description='Número 2:')

    boton_sumar = widgets.Button(description='Sumar')
    resultado = widgets.Output()

    pestana_entrada = widgets.VBox([numero1, numero2, boton_sumar])
    pestana_resultado = widgets.VBox([widgets.HTML("<b>Gráfica del Resultado:</b>"), resultado])

    def hacer_suma(b):
        with resultado:
            clear_output(wait=True)
            # Calcular suma
            suma = numero1.value + numero2.value

            # Datos para la gráfica
            categorias = ['Número 1', 'Número 2', 'Suma']
            valores = [numero1.value, numero2.value, suma]

            p = figure(x_range=categorias, height=300, title="Valores y su suma",
                       toolbar_location=None, tools="")

            p.vbar(x=categorias, top=valores, width=0.6, color=["blue", "green", "red"])

            p.y_range.start = 0
            p.xgrid.grid_line_color = None
            p.yaxis.axis_label = 'Valor'
            p.xaxis.axis_label = 'Categorías'

            show(p)

    boton_sumar.on_click(hacer_suma)

    tabs = widgets.Tab(children=[pestana_entrada, pestana_resultado])
    tabs.set_title(0, 'Entrada')
    tabs.set_title(1, 'Resultado')

    boton_cerrar = widgets.Button(description="❌ Cerrar ventana", button_style='danger')

    contenedor_ventana = widgets.VBox([tabs, boton_cerrar])

    def cerrar_ventana(b):
        salida_ventanas.children = tuple(child for child in salida_ventanas.children if child != contenedor_ventana)

    boton_cerrar.on_click(cerrar_ventana)

    return contenedor_ventana

def abrir_nueva_ventana(b):
    nueva_ventana = crear_ventana_con_tabs()
    salida_ventanas.children = list(salida_ventanas.children) + [nueva_ventana]
    contenedor_principal.children = [salida_ventanas, boton_nueva]

boton_nueva = widgets.Button(description="➕ Abrir nueva ventana", button_style='info')
boton_nueva.on_click(abrir_nueva_ventana)

contenedor_principal.children = [salida_ventanas, boton_nueva]
display(contenedor_principal)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from bokeh.plotting import figure
from bokeh.embed import components
from bokeh.resources import CDN

# Contenedor principal
contenedor_principal = widgets.VBox()
salida_ventanas = widgets.VBox()

def crear_ventana_con_tabs():
    numero1 = widgets.IntText(description='Número 1:')
    numero2 = widgets.IntText(description='Número 2:')

    boton_sumar = widgets.Button(description='Sumar')
    resultado = widgets.Output()

    pestana_entrada = widgets.VBox([numero1, numero2, boton_sumar])
    pestana_resultado = widgets.VBox([widgets.HTML("<b>Resultado:</b>"), resultado])

    def hacer_suma(b):
        with resultado:
            clear_output()
            suma = numero1.value + numero2.value
            print(f"La suma de {numero1.value} + {numero2.value} es {suma}")

            p = figure(title="Gráfica de ejemplo con Bokeh", x_axis_label='X', y_axis_label='Y')
            x = [1, 2, 3, 4, 5]
            y = [6, 7, 2, 4, 5]
            p.line(x, y, legend_label="Línea", line_width=2)
            p.scatter(x, y, size=8, color="red", marker="circle", legend_label="Puntos")

            # Obtener script y div
            script, div = components(p)

            # Obtener recursos JS y CSS (CDN)
            recursos = CDN.render()

            # Mostrar todo junto para que funcione
            display(HTML(recursos + div + script))

    boton_sumar.on_click(hacer_suma)
    #SON LOS APARTADOS QUE TENDRA LA VENTAN A
    tabs = widgets.Tab(children=[pestana_entrada, pestana_resultado])
    tabs.set_title(0, 'Entrada')
    tabs.set_title(1, 'Resultado')

    #BUTTON PARA CERRAR VENTANA
    boton_cerrar = widgets.Button(description="❌ Cerrar ventana", button_style='danger')

    contenedor_ventana = widgets.VBox([tabs, boton_cerrar])

    def cerrar_ventana(b):
        salida_ventanas.children = tuple(child for child in salida_ventanas.children if child != contenedor_ventana)

    boton_cerrar.on_click(cerrar_ventana)

    return contenedor_ventana

def abrir_nueva_ventana(b):
    nueva_ventana = crear_ventana_con_tabs()
    salida_ventanas.children = list(salida_ventanas.children) + [nueva_ventana]
    contenedor_principal.children = [salida_ventanas, boton_nueva]

boton_nueva = widgets.Button(description="➕ Abrir nueva ventana", button_style='info')
boton_nueva.on_click(abrir_nueva_ventana)

contenedor_principal.children = [salida_ventanas, boton_nueva]
display(contenedor_principal)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from bokeh.plotting import figure
from bokeh.embed import components
from bokeh.resources import CDN

# Contenedor principal
contenedor_principal = widgets.VBox()
salida_ventanas = widgets.VBox()

# ==============================
# Función para crear cada ventana
# ==============================
def crear_ventana_con_tabs(parametros_widget=None):
    numero1 = widgets.IntText(description='Número 1:')
    numero2 = widgets.IntText(description='Número 2:')
    boton_sumar = widgets.Button(description='Sumar')
    resultado = widgets.Output()

    # Pestaña 1: Entrada
    pestana_entrada = widgets.VBox([numero1, numero2, boton_sumar])

    # Pestaña 2: Resultado
    pestana_resultado = widgets.VBox([widgets.HTML("<b>Resultado:</b>"), resultado])

    # Lógica del botón
    def hacer_suma(b):
        with resultado:
            clear_output()
            suma = numero1.value + numero2.value
            print(f"La suma de {numero1.value} + {numero2.value} es {suma}")

            p = figure(title="Gráfica de ejemplo con Bokeh", x_axis_label='X', y_axis_label='Y')
            x = [1, 2, 3, 4, 5]
            y = [6, 7, 2, 4, 5]
            p.line(x, y, legend_label="Línea", line_width=2)
            p.scatter(x, y, size=8, color="red", marker="circle", legend_label="Puntos")

            script, div = components(p)
            recursos = CDN.render()
            display(HTML(recursos + div + script))

    boton_sumar.on_click(hacer_suma)

    # === Crear lista de pestañas ===
    pestañas = [pestana_entrada, pestana_resultado]
    titulos = ['Entrada', 'Resultado']

    # Pestaña 3: Parámetros personalizados (si se pasa uno)
    if parametros_widget:
        pestañas.append(parametros_widget)
        titulos.append('Parámetros')

    tabs = widgets.Tab(children=pestañas)
    for i, titulo in enumerate(titulos):
        tabs.set_title(i, titulo)

    # Botón cerrar ventana
    boton_cerrar = widgets.Button(description="❌ Cerrar ventana", button_style='danger')

    contenedor_ventana = widgets.VBox([tabs, boton_cerrar])

    def cerrar_ventana(b):
        salida_ventanas.children = tuple(child for child in salida_ventanas.children if child != contenedor_ventana)

    boton_cerrar.on_click(cerrar_ventana)

    return contenedor_ventana

# ==============================
# Función para abrir nueva ventana
# ==============================
def abrir_nueva_ventana(b):
    # Crear contenido para la pestaña "Parámetros"
    parametros = widgets.Output()
    with parametros:
        print("Parámetros personalizados de esta ventana:")
        print("- Gráfica con puntos rojos")
        print("- Línea azul simple")
        print("- Datos precargados")

    # Crear y mostrar nueva ventana
    nueva_ventana = crear_ventana_con_tabs(parametros)
    salida_ventanas.children = list(salida_ventanas.children) + [nueva_ventana]
    contenedor_principal.children = [salida_ventanas, boton_nueva]

# Botón principal para abrir nuevas ventanas
boton_nueva = widgets.Button(description="➕ Abrir nueva ventana", button_style='info')
boton_nueva.on_click(abrir_nueva_ventana)

# Mostrar interfaz principal
contenedor_principal.children = [salida_ventanas, boton_nueva]
display(contenedor_principal)


In [ ]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, CheckboxGroup, CustomJS, Button as BokehButton, CrosshairTool, HoverTool
from bokeh.layouts import row, column
from bokeh.embed import components
from bokeh.resources import CDN
from IPython.display import display, HTML
import numpy as np
import pandas as pd

# Datos simulados
x = pd.date_range(start='2023-01-01', periods=100)
y1 = np.sin(np.linspace(0, 10, 100))
y2 = y1 + np.random.normal(0, 0.1, 100)
y3 = np.where(np.random.rand(100) > 0.95, 2, np.nan)  # "Stuck" datos con pocos picos

source1 = ColumnDataSource(data=dict(x=x, y=y1))
source2 = ColumnDataSource(data=dict(x=x, y=y2))
source3 = ColumnDataSource(data=dict(x=x, y=y3))

# Crear figura
p = figure(width=700, height=400, x_axis_type='datetime', title="Gráfica Interactiva con Bokeh")
p.line('x', 'y', source=source1, line_width=2, color='blue', legend_label="Curva Original", name="line1")
line2 = p.line('x', 'y', source=source2, line_width=2, color='red', legend_label="Curva con Ruido", name="line2")
line3 = p.line('x', 'y', source=source3, line_width=2, color='black', legend_label="Stuck Data", name="line3")

# Herramientas Crosshair y Hover
p.add_tools(CrosshairTool(dimensions='both'))
p.add_tools(HoverTool(tooltips=[("Fecha", "@x{%F}"), ("Nivel", "@y")],
                     formatters={'@x': 'datetime'}, mode='vline'))

p.toolbar.logo = None
p.title.text_font_size = "20pt"
p.xaxis.axis_label = "Fecha"
p.xaxis.axis_label_text_font_size = "16pt"
p.yaxis.axis_label = "Nivel"
p.yaxis.axis_label_text_font_size = "16pt"

# Checkbox para mostrar/ocultar líneas
checkbox = CheckboxGroup(labels=["Mostrar curva original", "Mostrar curva con Ruido", "Mostrar Stuck Data"], active=[0,1,2])
checkbox.js_on_change('active', CustomJS(args=dict(l1=p.renderers[0], l2=p.renderers[1], l3=p.renderers[2], chk=checkbox), code="""
    l1.visible = chk.active.includes(0);
    l2.visible = chk.active.includes(1);
    l3.visible = chk.active.includes(2);
"""))

# Botones zoom X
x_in = BokehButton(label="< Zoom In X", width=100)
x_out = BokehButton(label="> Zoom Out X", width=100)
x_in.js_on_click(CustomJS(args=dict(p=p), code="""
    const range = p.x_range.end - p.x_range.start;
    const center = (p.x_range.start + p.x_range.end) / 2;
    const new_range = range * 0.8;
    p.x_range.start = center - new_range / 2;
    p.x_range.end = center + new_range / 2;
"""))
x_out.js_on_click(CustomJS(args=dict(p=p), code="""
    const range = p.x_range.end - p.x_range.start;
    const center = (p.x_range.start + p.x_range.end) / 2;
    const new_range = range * 1.2;
    p.x_range.start = center - new_range / 2;
    p.x_range.end = center + new_range / 2;
"""))

# Botones zoom Y
y_in = BokehButton(label="< Zoom In Y", width=100)
y_out = BokehButton(label="> Zoom Out Y", width=100)
y_in.js_on_click(CustomJS(args=dict(p=p), code="""
    const range = p.y_range.end - p.y_range.start;
    const center = (p.y_range.start + p.y_range.end) / 2;
    const new_range = range * 0.8;
    p.y_range.start = center - new_range / 2;
    p.y_range.end = center + new_range / 2;
"""))
y_out.js_on_click(CustomJS(args=dict(p=p), code="""
    const range = p.y_range.end - p.y_range.start;
    const center = (p.y_range.start + p.y_range.end) / 2;
    const new_range = range * 1.2;
    p.y_range.start = center - new_range / 2;
    p.y_range.end = center + new_range / 2;
"""))

# Layout
controles_x = row(x_in, x_out)
controles_y = row(y_in, y_out)
layout22 = row(checkbox, p)
layout_final = column(layout22, controles_x, controles_y)

# Obtener script y div
script, div = components(layout_final)

# Recursos CDN
recursos = CDN.render()

# Mostrar todo junto
html = recursos + div + script
display(HTML(html))


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files, drive
from ipywidgets import HBox, Button, widgets
from IPython.display import display, clear_output
##################################################
##################  NUEVAS IMPORTACIONES #########
##################################################
import datetime
import numpy as np
from os.path import basename
from pathlib import Path

from bokeh.plotting import figure
from bokeh.io import output_notebook,show

from bokeh.models import Button as BokehButton, HoverTool,CrosshairTool,DatetimeTickFormatter, LabelSet,ColumnDataSource,CheckboxGroup,CustomJS,Div
from scipy.interpolate import make_interp_spline
from bokeh.layouts import column,row
import hashlib

# Montar Drive si no está
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')
archivos_cargados=[]
ruta_drive = '/content/drive/MyDrive'
ruta_actual = [ruta_drive]
rutas_seleccionadas = []

salida_pc = widgets.Output()
salida_drive = widgets.Output()
salida_resultado = widgets.Output()

boton_pc = widgets.Button(description="💻 Usar archivo desde PC", button_style='info')
boton_drive = widgets.Button(description="📁 Navegar en Drive", button_style='success')

def mostrar_contenido_drive():
    with salida_drive:
        clear_output()
        actual = ruta_actual[0]
        archivos = os.listdir(actual)
        carpetas = sorted([f for f in archivos if os.path.isdir(os.path.join(actual, f))])
        ficheros = sorted([f for f in archivos if os.path.isfile(os.path.join(actual, f))])

        selector = widgets.SelectMultiple(
            options=carpetas + ficheros,
            description='Contenido:',
            rows=10,
            layout=widgets.Layout(width='80%')
        )

        boton_entrar = widgets.Button(description="📂 Entrar a carpeta")
        boton_atras = widgets.Button(description="⬅️ Atrás")
        boton_seleccionar = widgets.Button(description="✅ Seleccionar archivo(s)")

        def entrar_click(b):
            seleccion = list(selector.value)
            if len(seleccion) == 1:
                nombre = seleccion[0]
                nueva_ruta = os.path.join(actual, nombre)
                if os.path.isdir(nueva_ruta):
                    ruta_actual[0] = nueva_ruta
                    mostrar_contenido_drive()
                else:
                    with salida_resultado:
                        clear_output()
                        print("⚠️ Solo puedes entrar a una carpeta a la vez.")
            else:
                with salida_resultado:
                    clear_output()
                    print("⚠️ Solo puedes seleccionar una carpeta para entrar.")

        def atras_click(b):
            if ruta_actual[0] != ruta_drive:
                ruta_actual[0] = os.path.dirname(ruta_actual[0])
                mostrar_contenido_drive()

        def seleccionar_click(b):
            seleccionados = list(selector.value)
            agregados = 0
            for nombre in seleccionados:
                ruta_completa = os.path.join(actual, nombre)
                if os.path.isfile(ruta_completa):
                    if ruta_completa not in rutas_seleccionadas:
                        rutas_seleccionadas.append(ruta_completa)
                        agregados += 1
            mostrar_rutas()
            if agregados == 0:
                print("⚠️ Ningún archivo nuevo fue agregado.")

        boton_entrar.on_click(entrar_click)
        boton_atras.on_click(atras_click)
        boton_seleccionar.on_click(seleccionar_click)

        display(widgets.VBox([
            widgets.HTML(f"<b>Ruta actual:</b> {actual}"),
            selector,
            widgets.HBox([boton_atras, boton_entrar, boton_seleccionar])
        ]))

hashes_archivos = set()

def obtener_hash(archivo_path):
    hash_md5 = hashlib.md5()
    with open(archivo_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

def seleccionar_desde_pc(b=None):
    with salida_pc:
        clear_output()
        print("Sube un archivo desde tu PC:")
        upload = files.upload()

        for nombre in upload.keys():
            ruta_local = os.path.abspath(nombre)
            hash_archivo = obtener_hash(ruta_local)

            if hash_archivo not in hashes_archivos:
                rutas_seleccionadas.append(ruta_local)
                hashes_archivos.add(hash_archivo)
                print(f"✅ Archivo añadido: {os.path.basename(ruta_local)}")
            else:
                print(f"⚠️ Archivo duplicado (contenido igual): {os.path.basename(ruta_local)}")

        mostrar_rutas()
#####################################
def mostrar_rutas():
    with salida_resultado:
        clear_output()
        if rutas_seleccionadas:
            print("📂 Archivos seleccionados:")
            for i, ruta in enumerate(rutas_seleccionadas, 1):
                print(f"{i}. {ruta}")
                graficar_archivo(ruta)
        else:
            print("No se ha seleccionado ningún archivo aún.")

def activar_pc(b):
    salida_drive.clear_output()
    salida_resultado.clear_output()
    ruta_actual[0]=ruta_drive
    seleccionar_desde_pc(None)

def activar_drive(b):
    salida_pc.clear_output()
    salida_resultado.clear_output()
    ruta_actual[0] = ruta_drive
    mostrar_contenido_drive()

# Eventos
boton_pc.on_click(activar_pc)
boton_drive.on_click(activar_drive)

# Mostrar interfaz
display(widgets.HBox([boton_pc, boton_drive]))
display(salida_pc)
display(salida_drive)
display(salida_resultado)



def graficar_archivo(ruta_archivo):
  print(type(ruta_archivo))
  titulo2 = widgets.HTML("<h3 style='color: teal;'>Analisis</h3>")
  display(titulo2)
  # Mostrar interfaz inicial
  lista_archivos = []
  if ruta_archivo: # esto nos sirve para preguntar si se ha seleccionado un archivo
    lista_archivos.append(Path(ruta_archivo).name)

    for nombre_archivo in lista_archivos:
      ya_registrado = False # esto es nuestra bandera, para saber si el archivo a sido agregado

      for nombre22 in archivos_cargados:
        if nombre_archivo == nombre22:
          print("----------------------------------------------------------------------------------------")
          print(f"======  El archivo '{nombre_archivo}'ya ha se ha registrado")
          ya_registrado = True  # marcar como duplicado
          break

      if not ya_registrado:
        archivos_cargados.append(nombre_archivo)
        print("archivos cargados",archivos_cargados)
        matriz_toga = pd.read_csv(ruta_archivo,sep=r'\s+',names =["StationID", "StationName", "Date", "D1", "D2", "D3","D4", "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12"], engine = 'python', skiprows = 1, na_values = "9999")
        #print("Matriz toga",matriz_toga.head())
        # version mejorada de google colab

        display(matriz_toga.head(10)) # podemos ponerle valore al head(10) conforme a las filas que queremos ver
        lista_fecha= list()
        lista_datos=list()
        # nueva implementacion
        lista_etiquetas=list()
        lista_datos_no_picos=list()
        lista_fecha_no_picos=list()
        filas_dat=0
        #nueva implementacion con stuckdata
        stucktotal = 0
        stuckcount = 0
        stucklimit = 4

        stuckindex = 0

        lista_stuck_datos=list()
        lista_stuck_fechas=list()

        #incio con el analisis de los registros
        for filas_dat in matriz_toga.index:
          fecha = str(matriz_toga["Date"][filas_dat])
          if fecha[8:9]== "1":
            fecha_inicial = datetime.datetime(int(fecha[:4]),int(fecha[4:6]),int(fecha[6:8]))
          else:
            fecha_inicial = datetime.datetime(int(fecha[:4]),int(fecha[4:6]),int(fecha[6:8]),12)
          for dat in ["D1", "D2", "D3", "D4",  "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12"]:
            lista_fecha.append(fecha_inicial)
            lista_datos.append(matriz_toga[dat][filas_dat])
            #AGRAGMOS LAS LISTA ETIQUETAS
            lista_etiquetas.append(0)
            fecha_inicial=fecha_inicial+datetime.timedelta(hours=1)
        ######################################################################
          # =================== los datos sin pico  ================================
        filas_dat=0
        stuckvalue = lista_datos[0]

        for filas_dat in range(len(lista_datos)):
          if lista_etiquetas[filas_dat] !=8 and lista_etiquetas[filas_dat]!=3:
            lista_fecha_no_picos.append(lista_fecha[filas_dat])
            lista_datos_no_picos.append(lista_datos[filas_dat])

          ######################################################################
          # =================== los STUCKDATA  ================================
          if filas_dat >0:
            #verificamos si el dato que se esta revisando actualmente no es igual stuckvalue
            if stuckvalue == lista_datos[filas_dat] and filas_dat -1 == stuckindex:
              stuckcount = stuckcount +1
            else:
              stuckcount = 0
            #se comple cuando cuando stuckvalue sea igual a lista_datos y que stuklimit sea menor que stuckcount
            if stuckcount >= stucklimit-1:

              # si se trata de los primeros tres valores, etiquetar los dos valores que siguen
              if stuckcount == stucklimit-1:
                for k in range(filas_dat-(stucklimit-1),filas_dat):
                  lista_etiquetas[k]=3
                  #print("--se han encontrado valores iguales consecutivos (stuck values).valor:" + str(lista_datos[k]) + "posicion:" + str(k))
                  lista_stuck_datos.append(lista_datos[k])
                  lista_stuck_fechas.append(lista_fecha[k])
                stucktotal = stucktotal + 1
              #etiqueta el stuckvalue actual
              lista_etiquetas[filas_dat]=3
              #print("-- se ha encontrado y etiquetado valores iguales consecutivos (stuck values).valor:" + str(lista_datos[filas_dat]) + "posicion:" + str(filas_dat))
              lista_stuck_datos.append(lista_datos[filas_dat])
              lista_stuck_fechas.append(lista_fecha[filas_dat])
            #si no pasa nada solo se le asigna el valor a stuckvalue
            stuckvalue = lista_datos[filas_dat]
            stuckindex = filas_dat

        #crear herramientas

        herramientas=[
            "pan",
            "wheel_zoom",
            "box_zoom",
            "reset",
            "save",
            "lasso_select",
            "box_select",
            "tap",
            "zoom_in",
            "zoom_out",
            "help"
        ]
        p = figure(
            title=f"Gráfica del archivo Toga: {lista_archivos}",
            x_axis_label="Fecha",
            y_axis_label="Nivel del mar (mm)",
            x_axis_type="datetime",
            width=1000,
            height=600,
            tools=','.join(herramientas),
            toolbar_location="above",
            toolbar_sticky=False,
            margin=(50, 50, 50, 50)
        )
        #Añadir HoverTool(muestra tooltip con datos exactos al pasar el mouse)
        hover = HoverTool(
            tooltips=[("Fecha", "@x{%F}"), ("Nivel", "@y")],
            formatters={'@x': 'datetime'},
            mode='vline'
        )

        #Añadir CrosshairTool (Muestra lineas guia horizontal y verticales para facilitar lectura)
        crosshair = CrosshairTool(dimensions= 'both')#lineas vertical y horizontal
        p.add_tools(crosshair)

        #creamos la grafica de diferente forma
        # NUESTRO CODIGO DE ZOOM INTERACTIVO
        source1 = ColumnDataSource(data=dict(x=lista_fecha,y=lista_datos))
        source2= ColumnDataSource(data=dict(x=lista_fecha_no_picos,y=lista_datos_no_picos))
        source3= ColumnDataSource(data=dict(x=lista_stuck_fechas,y=lista_stuck_datos))


        x_in = BokehButton(label="<Zoom In X",width=100)
        x_out= BokehButton(label=">Zoom Out X",width=100)

        x_in.js_on_click(CustomJS(args=dict(p=p),code="""
            let range = p.x_range.end - p.x_range.start;
            let center = (p.x_range.start + p.x_range.end) /2;
            let new_range = range * 0.8;
            p.x_range.start =center-new_range/2;
            p.x_range.end = center + new_range/2;
        """))

        x_out.js_on_click(CustomJS(args=dict(p=p),code="""
            let range = p.x_range.end - p.x_range.start;
            let center = (p.x_range.start + p.x_range.end)/2;
            let new_range = range * 1.2;
            p.x_range.start = center -new_range/2;
            p.x_range.end = center + new_range/2;
        """))

        #Botones para Y
        y_in=BokehButton(label="<zoom In Y",width=100)
        y_out=BokehButton(label=">Zoom Out Y",width=100)

        y_in.js_on_click(CustomJS(args=dict(p=p),code="""
            let range = p.y_range.end - p.y_range.start;
            let center = (p.y_range.start + p.y_range.end)/2;
            let new_range = range * 0.8;
            p.y_range.start = center -new_range/2;
            p.y_range.end = center + new_range/2;
        """))

        y_out.js_on_click(CustomJS(args=dict(p=p),code="""
            let range = p.y_range.end - p.y_range.start;
            let center = (p.y_range.start + p.y_range.end)/2;
            let new_range = range *1.2;
            p.y_range.start = center - new_range/2;
            p.y_range.end = center + new_range / 2;
        """))

        #crear la figura
        linea1 = p.line(source=source1, line_width=2, color="blue")
        linea2 = p.line(source=source2, line_width=2, color="red")
        linea3 = p.line(source=source3, line_width=2, color="black")
        #nuestro layout final para el ZOOM INTERACTIVO
        controles_x = row(x_in,x_out)
        controles_y = row(y_in,y_out)

        #checkbox para mostrar/ ocultar
        checkbox = CheckboxGroup(labels=["Mostrar curva original","Mostrar curva con Ruido","Mostrar StuckData"],active=[0])
        #inicializamos la visibilidad al ejecutar
        linea1.visible=0 in checkbox.active
        linea2.visible= 1 in checkbox.active
        linea3.visible= 2 in checkbox.active

        #JS para apagar /encender lineas
        callback = CustomJS(args=dict(l1=linea1,l2=linea2,l3=linea3,chk=checkbox),code="""
            l1.visible=chk.active.includes(0);
            l2.visible=chk.active.includes(1);
            l3.visible=chk.active.includes(2);
        """)
        #instruccion que detecta el cambio y lo asocia al callback
        checkbox.js_on_change('active',callback)
        p.title.text_font_size = "20pt"
        p.xaxis.axis_label_text_font_size = "16pt"
        p.yaxis.axis_label_text_font_size = "16pt"
        p.add_tools(HoverTool(tooltips=[("Fecha", "@x{%F}"), ("Nivel", "@y")], formatters={'@x': 'datetime'}))
        p.toolbar.logo = None  # Ocultar logo de Bokeh

        layout22 = row(checkbox,p)
        show(column(layout22,controles_x,controles_y))

        print("llegada fffff")
  else:
    print("No se ha seleccionado ningún archivo.")



In [5]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from bokeh.plotting import figure
from bokeh.embed import components
from bokeh.resources import CDN

# === Valor compartido y salida gráfica ===
valor_actual = widgets.IntText(value=0, description="Valor:", disabled=True)
salida_grafica = widgets.Output()

# === Función para graficar automáticamente ===
def graficar(valor):
    salida_grafica.clear_output()
    with salida_grafica:
        print(f"📊 Graficando: {valor}")
        x = [1, 2, 3, 4, 5]
        y = [i * valor for i in x]

        p = figure(title=f"y = x * {valor}", x_axis_label='X', y_axis_label='Y')
        p.line(x, y, line_width=2, legend_label="y = x * valor")
        p.scatter(x, y, size=8, color="blue")

        script, div = components(p)
        display(HTML(CDN.render() + div + script))

# === Botones para modificar el valor ===
boton3 = widgets.Button(description="Enviar 3", button_style='success')
boton7 = widgets.Button(description="Enviar 7", button_style='info')
salida_accion = widgets.Output()

def enviar_valor(valor):
    valor_actual.value = valor
    with salida_accion:
        clear_output()
        print(f"📤 Valor enviado: {valor}")

boton3.on_click(lambda b: enviar_valor(3))
boton7.on_click(lambda b: enviar_valor(7))

# === Pestaña 1: Entrada ===
pestana_entrada = widgets.VBox([
    widgets.HTML("<b>Selecciona un número:</b>"),
    widgets.HBox([boton3, boton7]),
    valor_actual,
    salida_accion
])

# === Pestaña 2: Gráfica ===
pestana_grafica = widgets.VBox([
    widgets.HTML("<b>Gráfica generada:</b>"),
    salida_grafica
])

# === Tabs principales ===
tabs = widgets.Tab(children=[pestana_entrada, pestana_grafica])
tabs.set_title(0, "Entrada")
tabs.set_title(1, "Gráfica")

# === Detectar cambio de pestaña ===
def cambio_de_pestana(change):
    if change['new'] == 1:  # Si cambia a la pestaña "Gráfica"
        graficar(valor_actual.value)

tabs.observe(cambio_de_pestana, names='selected_index')

# === Mostrar la interfaz ===
display(widgets.HTML("<h3>Ventana Unificada: Entrada y Gráfica</h3>"))
display(tabs)


HTML(value='<h3>Ventana Unificada: Entrada y Gráfica</h3>')

In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from bokeh.plotting import figure
from bokeh.embed import components
from bokeh.resources import CDN
############################
##============= 1  ===========h
################################
# === VALORES Y SALIDAS COMPARTIDAS ===
valor_actual = widgets.IntText(value=0, description="Valor:", disabled=True)
salida_resultado = widgets.Output()
salida_pc = widgets.Output()
salida_drive = widgets.Output()
salida_grafica = widgets.Output()

# === FUNCIONES PERSONALIZADAS ===
############################
##============= 2  ===========
################################
def activar_pc(b=None):
    valor_actual.value = 3  # Valor relacionado con PC
    with salida_pc:
        clear_output()
        print("🖥️ PC activada")
    with salida_resultado:
        clear_output()
        print("📤 Valor enviado desde PC: 3")
############################
##============= 3  ===========f
################################
def activar_drive(b=None):
    valor_actual.value = 7  # Valor relacionado con Drive
    with salida_drive:
        clear_output()
        print("💾 Drive activado")
    with salida_resultado:
        clear_output()
        print("📤 Valor enviado desde Drive: 7")
############################
##============= 4  ===========
################################
# === GRAFICAR CON BOKEH ===
def graficar(valor):
    salida_grafica.clear_output()
    with salida_grafica:
        print(f"📊 Graficando con valor: {valor}")
        x = [1, 2, 3, 4, 5]
        y = [i * valor for i in x]

        p = figure(title=f"y = x * {valor}", x_axis_label='X', y_axis_label='Y')
        p.line(x, y, line_width=2, legend_label="y = x * valor")
        p.scatter(x, y, size=8, color="green")

        script, div = components(p)
        display(HTML(CDN.render() + div + script))
############################
##============= 5  ===========h
################################
# === BOTONES PERSONALIZADOS ===
boton_pc = widgets.Button(description="Activar PC", button_style='success')
boton_drive = widgets.Button(description="Activar Drive", button_style='info')
############################
##============= 6  ===========h
################################
boton_pc.on_click(activar_pc)
boton_drive.on_click(activar_drive)
############################
##============= 7  ===========h
################################
# === PESTAÑA 1: Entrada ===
pestana_entrada = widgets.VBox([
    widgets.HTML("<b>Acciones disponibles:</b>"),
    widgets.HBox([boton_pc, boton_drive]),
    valor_actual,
    salida_pc,
    salida_drive,
    salida_resultado
])
############################
##============= 8  ===========h
################################
# === PESTAÑA 2: Gráfica ===
pestana_grafica = widgets.VBox([
    widgets.HTML("<b>Gráfica generada:</b>"),
    salida_grafica
])
############################
##============= 9  ===========p
################################
# === TABS CON DETECTOR ===
tabs = widgets.Tab(children=[pestana_entrada, pestana_grafica])
tabs.set_title(0, "Entrada")
tabs.set_title(1, "Gráfica")
############################
##============= 10  ===========h
################################
def cambio_de_pestana(change):
    if change['new'] == 1:  # Entró a la pestaña de Gráfica
        graficar(valor_actual.value)
############################
##============= 11  ===========h
################################
tabs.observe(cambio_de_pestana, names='selected_index')
############################
##============= 12  ===========h
################################
# === MOSTRAR VENTANA COMPLETA ===
display(widgets.HTML("<h3>Ventana Unificada con Acciones y Gráfica</h3>"))
display(tabs)


HTML(value='<h3>Ventana Unificada con Acciones y Gráfica</h3>')